# Exercise: Climate

#### Summary

Do nothing very useful with climate data from the US government.

#### Data Source(s)

Data from NOAA, https://www7.ncdc.noaa.gov/CDO/CDODivisionalSelect.jsp

#### Files

- CDODiv8449537379627.txt, climate data from 1985-2017
- florida.txt, california.txt, virginia.txt, minnesota.txt, nebraska.txt, same as above but for specific states

#### Data dictionary

- **PCP** - Precipitation Index
- **TAVG** - Temperature Index
- **TMIN** - Minimum Temperature Index
- **TMAX** - Maximum Temperature Index
- **PDSI** - Palmer Drought Severity Index
- **PHDI** - Palmer Hydrological Drought Index
- **ZNDX** - Palmer Z-Index
- **PMDI** - Modified Palmer Drought Severity Index
- **CDD** - Cooling Degree Days
- **HDD** - Heating Degree Days
- **SPnn** - Standard Precipitation Index

#### Skills

- Cleaning column names
- NaN values
- Date time manipulation
- Plotting

# Reading in your data

## Open up the file and check the first few rows

In [2]:
import pandas as pd
import numpy as np
import re
import statistics
%matplotlib inline

In [9]:
!pwd

/Users/wangchen/Desktop/Lede-Summer/Foundation/Foundation/class-12


In [7]:
df = pd.read_csv('data/climate/CD', na_values=['-99.99', '-99.90', '-9.99', '-9999'])
pd.set_option('display.max_columns', 65)
df.head()

FileNotFoundError: File b'data/climate/CDODiv8449537379627.txt' does not exist

## Hmmm, `-99.99` doesn't seem like good data!

And if you read [the documentation](ftp://ftp.ncdc.noaa.gov/pub/data/cirs/climdiv/divisional-readme.txt), you find out it means MISSING DATA. Get rid of those -99.99s however you think is best.

In [ ]:
# solved in last cell by using na_values=[]

## Wait, there's even more missing data!

Open up [the documentation](ftp://ftp.ncdc.noaa.gov/pub/data/cirs/climdiv/divisional-readme.txt) and look for the other values that are "missing."

- Tip: don't read all of the documentation, use the power of searching!

In [ ]:
# ftp://ftp.ncdc.noaa.gov/pub/data/cirs/climdiv/divisional-readme.txt
# solved in last cell by using na_values=[]

## Confirm that you have 1465 rows and 21 columns

In [ ]:
df.shape

## Let's see the columns and their data types

In [ ]:
df.dtypes

## Uuuuhhh something is weird

Try to get the average `TMAX`. It *probably* will not work. Why not?

In [ ]:
df['TMAX'].mean

## Cleaning up

### Fix the column names!

Right now TMAX (and the rest of the column names) are looking pretty bad. Let's turn them into normal columns! **If you couldn't figure out what's wrong with them, ask!**

- Tip: You can get a list of the columns by using `df.columns`.
- Tip: You can actually treat them just like a pandas column
- Tip: What would you use if you had to remove whitespace (spaces) from the left and right of a column in pandas?
- Tip: Instead of using rename, you can set the columns to something new with `df.columns = blah blah blah` (but be sure to **check that your edited ones are right** before you assign them)

In [ ]:
df.columns

In [ ]:
df.columns = pd.Series([column.strip() for column in df.columns])
df.columns

### Find the hottest months recorded

Just to make sure you cleaned it all up correctly - if you didn't clean the column titles, this won't work.

In [ ]:
df['TMAX'].sort_values(ascending=False).head()

## That "Unnamed" column doesn't actually do anything. Delete it!

One of the things we talked about last class was **creating new dataframes** instead of editing our old ones. So we *could* do this:

```python
df = df[['col1', 'col2', 'col2', 'col3']]
```

**But don't do that!** I don't want you to type out all of those columns. I just want you to delete that column.

In [ ]:
df.drop(labels='Unnamed: 20', axis=1, inplace=True)

In [ ]:
df.head()

## What is the `YearMonth` column?

What is its data type, and what is the content in it?

In [ ]:
df['YearMonth'].dtypes

## Convert `YearMonth` into a date (don't save it anywhere yet)

**Be sure to pass a `format=` to the converter.** It will work automatically, but it's better to get practice.

If you convert it correctly, it should look like...

    0      1895-01-01
    1      1895-02-01
    2      1895-03-01
    3      1895-04-01
    ...

In [ ]:
pd.to_datetime(df['YearMonth'], format='%Y%m').head()

## Save this converted version into a new column called `Date`

In [ ]:
df['Date'] = pd.to_datetime(df['YearMonth'], format='%Y%m')
df['Date']

### What is the datatype of the `Date` columns?

In [ ]:
df.dtypes

# Graphing

## Graph the maximum temperature over time using the old `YearMonth` column

Make it so wide it **extends across your entire screen**.

In [ ]:
df['TMAX'].plot(x='YearMonth', figsize=(17,5))

In [ ]:
df.plot(x='YearMonth', y='TMAX', figsize=(17,5))

## Graph the maximum temperature over time using the new `Date` column

Make it so wide it **extends across your entire screen**.

In [ ]:
df.plot(x='Date', y='TMAX', figsize=(17,5))

## How do they look different? Which one is better, and why?

In [ ]:
# The second one is better. In the first graph, all the numbers of one year are displayed in a spike and then jump to the next year.

# Resampling

Check that you have pandas 0.19.0 or newer by typing `pd.__version__`. If you have an earlier version, use `pip install -U pandas` to update.

In [ ]:
pd.__version__

## View the average yearly temperature

**Don't use `.set_index` yet.** When using `resample`, you can also say `on='colname'` in order to say "resample using this column instead of using the index. Use that here.

In [ ]:
df.resample('A', on='Date')['TAVG'].mean()

## Graph the average annual temperature

Be sure ot make it **nice and wide**.

In [ ]:
df.resample('A', on='Date')['TAVG'].mean().plot(figsize=(17,5))

## What's wrong with the right-hand side?

Use `.set_index` to get a datetime index and then examine the months in 1990 and the months in 2017 and see what's different about them.

In [ ]:
df = df.set_index('Date')

In [ ]:
df['1990'].count()

In [ ]:
df['2017'].count()

## Upgrading to good data

**We can ignore 2017.** Edit our dataframe so it's only between the years of 1895 and 2016, and draw your graph again.

In [ ]:
df['1895':'2016'].resample('A')['TAVG'].mean().plot(figsize=(17,5))

### Maxes and mins

What years had the hottest maximum temperature? The hottest minimum temperature?

In [ ]:
df['1895':'2016'].resample('A')['TMAX'].max().sort_values(ascending=False).head(1)

In [ ]:
df['1895':'2016'].resample('A')['TMIN'].max().sort_values(ascending=False).head(1)

## Annual temperature cycles

Graph the average temperature cycle of a year, with january on the left and december on the right.

In [ ]:
df['1895':'2016'].groupby(df['1895':'2016'].index.month)['TAVG'].mean().plot()

# An important investigation

Inside of the `climate` folder there is a file called `california.txt`: it's the same data, but only for California. Using this data, tell me **if droughts have been getting worse over the past 100 years.**

In [ ]:
df_ca = pd.read_csv('data/climate/california.txt', na_values=['-99.99', '-99.90', '-9.99', '-9999'])
df_ca.head()

In [ ]:
df_ca.columns = pd.Series([column.strip() for column in df_ca.columns])
df_ca.columns

In [ ]:
df_ca['Date'] = pd.to_datetime(df_ca['YearMonth'], format='%Y%m')
df_ca.set_index('Date', inplace=True)

In [ ]:
df_ca.head()

In [ ]:
df_ca.resample('A')['PDSI'].mean().plot(figsize=(17,5))

# Making comparisons

There are datasets for **five** other states. Please compare them in some way, building me a graphic. Even though we did it the other day, you do **not** need to combine them into one dataframe.

Some basic ideas:

- Graph their average temperatures over the year so we can see which one is the hottest
- Graph the difference between the highs/lows over the year so we can see which has the hottest temperature swings
- Graph their propensity to drought, highlighting the most drough-ridden state in a different color

In [ ]:
import glob

In [ ]:
filenames = glob.glob("data/climate/*.txt")

In [ ]:
list_of_dfs = [pd.read_csv(filename, na_values=['-99.99', '-99.90', '-9.99', '-9999']) for filename in filenames]

In [ ]:
for dataframe, filename in zip(list_of_dfs, filenames):
    dataframe['filename'] = filename

In [ ]:
df = pd.read_csv('data/climate/CDODiv8449537379627.txt', na_values=['-99.99', '-99.90', '-9.99', '-9999'])
pd.set_option('display.max_columns', 65)

In [ ]:
list_of_dfs[3]

In [ ]:
for df in list_of_dfs:
    if df['filename'].isin(['data/climate/colorado.txt', 'data/climate/florida.txt', 'data/climate/louisiana.txt', 'data/climate/minnesota.txt', 'data/climate/virginia.txt']).any():
        df.columns = df.columns.str.strip()
        df['Date'] = pd.to_datetime(df['YearMonth'], format='%Y%m')
        df.set_index('Date', inplace=True)
        df.groupby(df.index.month)['TAVG'].mean().plot(figsize=(17,5))